In [1]:
conferences_by_id={
    "NeurIPS":1127325140,
    "ICML":1180662882,
    "ICLR":2584161585,
    "AAAI":1184914352,
    "ACL":1188739475,
    "CVPR":1158167855,
    "AISTATS":2622962978,
    "KDD":1130985203,
    "IJCAI":1203999783,
    "SIGIR":1140684652,
}

In [51]:
from unidecode import unidecode
import string
import pandas as pd
from gensim.utils import deaccent

def clean_title_for_MAG(link_file):
    #print(link_file).columns
    #link_file=link_file[~pd.isnull(link_file['PaperTitle'])].reset_index()
    link_file['PWC_Clean_Title']=link_file['title'].str.lower().str.replace('_',"UNDERSCORE").\
    str.replace('–',' ').str.replace('—',' ').str.replace('\'',' ').replace(u'’', ' ').str.replace(u'\u0027',' ').str.replace(u'\u2019',' ').\
    str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf8').\
    str.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))).\
    str.replace(' +', ' ',regex=True).str.replace("UNDERSCORE","_").str.strip()
    #str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf8').\
    link_file['PWC_Clean_Title']=link_file['PWC_Clean_Title'].apply(lambda x: deaccent(str(x)))
    link_file['PWC_Clean_Title']=link_file['PWC_Clean_Title'].str.replace('”', "",regex=True).str.replace('“', "",regex=True).\
    str.replace('−', " ",regex=True).str.replace('‐', " ",regex=True).\
    str.replace('―', "",regex=True).str.replace('ø', "o",regex=True).str.replace("â","a",regex=True).\
    str.replace('ℓ', "l",regex=True).str.replace("  "," ").str.replace("ε","e")
    return link_file

In [37]:
import pandas as pd
import json

pwc=pd.read_json('../Dataset_Curation/PWC_Data/papers-with-abstracts.json')

benchmark_tables=pd.read_json('../Dataset_Curation/PWC_Data/Derivative_Datasets/benchmarks_with_datasets.json')
benchmark_tables=benchmark_tables.rename({'paper_title':'title'},axis=1)

datasets=pd.read_json('../Dataset_Curation/PWC_Data/datasets.json')
datasets['title']=datasets['paper'].apply(lambda js: js['title'] if js is not None else None)
dataset_citing_papers=pd.read_csv('../Dataset_Curation/PWC_Data/Derivative_Datasets/datasets_citing_papers.txt',sep='\t')

In [52]:
pwc_titles=pd.concat([pwc['title'],benchmark_tables['title'],
                      datasets['title'],dataset_citing_papers['title']]).drop_duplicates().to_frame()
pwc_titles=pwc_titles[~pwc_titles.isnull()]
pwc_titles=clean_title_for_MAG(pwc_titles)

In [88]:
all_conf_titles=[]
all_conf_disp_titles=[]
all_conf_cc=[]
all_conf_y=[]

for c in conferences_by_id:
    with open(f'./conferences/{c}.json') as f:
        mag_papers=json.load(f)
    if mag_papers['expr'].find(str(conferences_by_id[c]))==-1: print("WRONG CONFERENCE")
    all_conf_titles+=[e['Ti'] for e in mag_papers['entities'] if e['Y']>=2015 and e['Y']<=2020]
    all_conf_disp_titles+=[e['DN'] for e in mag_papers['entities'] if e['Y']>=2015 and e['Y']<=2020]
    all_conf_cc+=[e['CC'] for e in mag_papers['entities'] if e['Y']>=2015 and e['Y']<=2020]
    all_conf_y+=[e['Y'] for e in mag_papers['entities'] if e['Y']>=2015 and e['Y']<=2020]
    
print(len(all_conf_titles),len(all_conf_disp_titles),len(all_conf_cc),len(all_conf_y))

46774 46774 46774 46774


In [89]:
mag_papers=pd.DataFrame({'title':all_conf_titles,'display_title':all_conf_disp_titles,'citations':all_conf_cc,'year':all_conf_y})

In [93]:
matches=(mag_papers['title'].isin(pwc_titles['PWC_Clean_Title']) | mag_papers['display_title'].isin(pwc_titles['title']))
print("Total number of papers: ", mag_papers.shape[0])
print(matches.sum()/mag_papers.shape[0], " percent of top conference papers are in PWC.")
print(mag_papers['citations'][matches].sum()/mag_papers['citations'].sum(), " percent of top conference citations are in PWC.")

for y in range(2015,2021):
    print("Year: ",y)
    mag_papers_y=mag_papers[mag_papers['year']==y]
    matches=(mag_papers_y['title'].isin(pwc_titles['PWC_Clean_Title']) | mag_papers_y['display_title'].isin(pwc_titles['title']))
    print("Total number of papers: ", mag_papers_y.shape[0])
    print(matches.sum()/mag_papers_y.shape[0], " percent of top conference papers are in PWC.")
    print(mag_papers_y['citations'][matches].sum()/mag_papers_y['citations'].sum(), " percent of top conference citations are in PWC.")


Total number of papers:  46774
0.5894514046265019  percent of top conference papers are in PWC.
0.893462666006669  percent of top conference citations are in PWC.
Year:  2015
Total number of papers:  3881
0.3893326462251997  percent of top conference papers are in PWC.
0.8647727691667588  percent of top conference citations are in PWC.
Year:  2016
Total number of papers:  5452
0.48826118855465883  percent of top conference papers are in PWC.
0.8948825289800354  percent of top conference citations are in PWC.
Year:  2017
Total number of papers:  6577
0.5298768435456895  percent of top conference papers are in PWC.
0.9000993945965483  percent of top conference citations are in PWC.
Year:  2018
Total number of papers:  7907
0.5997217655242191  percent of top conference papers are in PWC.
0.9012898877093004  percent of top conference citations are in PWC.
Year:  2019
Total number of papers:  10002
0.6254749050189962  percent of top conference papers are in PWC.
0.8956805336792365  percent 